In [1]:
# run dependencies
# !pip install xgboost
# !pip install shap

import pandas as pd
import numpy as np
import seaborn as sn
from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

# import xgboost
# import shap

import matplotlib.pyplot as plt

In [2]:
# CSV FILES
player_appearances_csv = "Data/player_appearances.csv"
players_csv = "Data/players.csv"
matches_csv = "Data/matches.csv"
data2021_csv = "Data/Players_EAFIFA/2021.csv"

In [3]:
# READ CSV FILES
playerApps_df = pd.read_csv(player_appearances_csv)
player_df = pd.read_csv(players_csv)
data_df = pd.read_csv(data2021_csv)
matches_df = pd.read_csv(matches_csv)

# CLEAN UP 
playerApps_df["Name"] = playerApps_df["given_name"] + " " + playerApps_df["family_name"]

dict = {'Overal': 'Overall',
        'Natinality': 'Country'}
data_df.rename(columns=dict,inplace = True)


pd.set_option('display.max_columns', None)

In [4]:
# CREATE GROUPED DATAFRAME SHOWING LIFETIME NUMBER OF WORLD CUP MATCHES 
playersApps_df_grp = playerApps_df[["match_id", "player_id", "Name"]].\
                            groupby(["player_id", "Name"]).nunique("match_id")
# playersApps_df_grp

In [5]:
# IDENTIFYING 2022 WORLD CUP QUALIFYING TEAMS
teams_2022 = ['Qatar', 'Netherlands', 'Senegal', 'Ecuador', 
            'England', 'United States', 'Wales', 'Iran', 
            'Argentina', 'Poland', 'Mexico', 'Saudi Arabia', 
            'France', 'Denmark', 'Tunisia', 'Australia', 
            'Germany', 'Spain', 'Japan', 'Costa Rica', 
            'Belgium', 'Croatia', 'Canada', 'Morocco', 
            'Brazil', 'Switzerland', 'Serbia', 'Cameroon', 
            'Portugal', 'Uruguay', 'Ghana', 'Korea Republic']

In [6]:
# MERGING DATAFRAMES

# MERGE PLAYER APPEARNACES WITH PLAYER DATA TO GET ADDITIONAL DETAIL PER PLAYER
player_info_df = pd.merge(playerApps_df, player_df, on = "player_id", how= "inner")

# MERGE LIFETIME NUMBER OF WORLD CUP MATCHES ONTO PLAYER RECORDS FROM FIFA 2021 PLAYER SET
df_wc = pd.merge(data_df,playersApps_df_grp, on="Name", how="left")
df_wc = df_wc.rename(columns={"match_id":"Lifetime_WC_Matches"})

# ---------- ADDING COLUMNS TO SERVE AS TARGETS: ----------
# LIFETIME WORLD CUP APPEARANCES (TOTAL NUMBERS)
df_wc.loc[df_wc["Lifetime_WC_Matches"].notnull(), "Lifetime_WC_Matches_Bin"]=1
# ----------
# BINARY: WHERE LIFETIME MATCHES IS MORE THAN 3 
    # (INDICATES ADVANCING THROUGH THE GROUP STAGE IF PLAYED IN ONE YEAR'S WORLD CUP \
    # OR MULTIPLE, SINGLE APPEARNACES AS QUALIFYING TEAM)
df_wc.loc[df_wc["Lifetime_WC_Matches"]>3, "Lifetime_WC_Matches_Bin4"]=1

# ----------
# BINARY: WHERE LIFETIME MATCHES IS MORE THAN 4 
df_wc.loc[df_wc["Lifetime_WC_Matches"]>4, "Lifetime_WC_Matches_Bin5"]=1

# ----------
# BINARY: WHERE LIFETIME MATCHES IS AT LEAST 9 (APPX MIDPOINT OF LIFETIME WC MATCHES VALUE COUNTS)
df_wc.loc[df_wc["Lifetime_WC_Matches"]>=9, "Lifetime_WC_Matches_Bin10"]=1

# df_wc.info()

In [7]:
# VIEW DATAFRAMES

# playerApps_df.head()
# player_info_df.head()
# matches_df.head()
# data_df.head()
# df_wc.head()
player_df.head()

,key_id,player_id,family_name,given_name,birth_date,goal_keeper,defender,midfielder,forward,count_tournaments,list_tournaments,player_wikipedia_link
0,1,P-05074,A'Court,Alan,1934-09-30,0,0,0,1,1,1958,https://en.wikipedia.org/wiki/Alan_A%27Court
1,2,P-00942,Abadzhiev,Stefan,1934-07-03,0,0,0,1,1,1966,https://en.wikipedia.org/wiki/Stefan_Abadzhiev
2,3,P-03051,Abalo,Jean-Paul,1975-06-26,0,1,0,0,1,2006,https://en.wikipedia.org/wiki/Jean-Paul_Abalo
3,4,P-03371,Abanda,Patrice,1978-08-03,0,1,0,0,1,1998,https://en.wikipedia.org/wiki/Patrice_Abanda
4,5,P-04977,Abate,Ignazio,1986-11-12,0,1,0,0,1,2014,https://en.wikipedia.org/wiki/Ignazio_Abate


In [8]:
# # iterating the columns
# for col in player_info_df.columns:
#     print(col)

print("PLAYER_INFO_DF COLUMNS")
print(player_info_df.columns)
print("-"*50)
print("MATCHES_DF COLUMNS")
print(matches_df.columns)
print("-"*50)
print("PLAYERAPPS_DF")
print(playerApps_df.columns)

PLAYER_INFO_DF COLUMNS
Index(['key_id_x', 'tournament_id', 'tournament_name', 'match_id',
       'match_name', 'match_date', 'stage_name', 'group_name', 'team_id',
       'team_name', 'team_code', 'home_team', 'away_team', 'player_id',
       'family_name_x', 'given_name_x', 'shirt_number', 'position_name',
       'position_code', 'starter', 'substitute', 'captain', 'Name', 'key_id_y',
       'family_name_y', 'given_name_y', 'birth_date', 'goal_keeper',
       'defender', 'midfielder', 'forward', 'count_tournaments',
       'list_tournaments', 'player_wikipedia_link'],
      dtype='object')
--------------------------------------------------
MATCHES_DF COLUMNS
Index(['key_id', 'tournament_id', 'tournament_name', 'match_id', 'match_name',
       'stage_name', 'group_name', 'group_stage', 'knockout_stage', 'replayed',
       'replay', 'match_date', 'match_time', 'stadium_id', 'stadium_name',
       'city_name', 'country_name', 'home_team_id', 'home_team_name',
       'home_team_code', 'aw

In [9]:
# df_wc.info()

# THE NUMBER OF PLAYERS PER COUNTRY BY LIFETIME WORLD CUP APPEARANCE BIN
player_ct_country_wcMatches_beta = df_wc[["Country", 
                                          "ID", 
                                          "Lifetime_WC_Matches_Bin", 
                                          "Lifetime_WC_Matches_Bin4", 
                                          "Lifetime_WC_Matches_Bin5", 
                                          "Lifetime_WC_Matches_Bin10"]].groupby("Country").count()

fifa2021_playerct = df_wc[["Country", "ID"]].groupby("Country").count().sort_values("ID", ascending = False)
# fifa2021_playerct

player_ct_country_wcMatches.sort_values("Lifetime_WC_Matches_Bin10", ascending = False).head(50)

player_ct_country_wcMatches = player_ct_country_wcMatches.rename(columns = {"ID_x": "Total FIFA 2021 Players", 
                                              "Lifetime_WC_Matches_Bin": "Players w/ Any WC Apperances",
                                              "Lifetime_WC_Matches_Bin4": "Players w/ 4+ WC Apperances",
                                              "Lifetime_WC_Matches_Bin5": "Players w/ 5+ WC Apperances",
                                              "Lifetime_WC_Matches_Bin19": "Players w/ 9+ WC Apperances"})

player_ct_country_wcMatches
# player_ct_country_wcMatchesc.loc[player_ct_country_wcMatches["Lifetime_WC_Matches"]>3, "Lifetime_WC_Matches_Bin4"]=1

NameError: name 'player_ct_country_wcMatches' is not defined

* Create a model to predict which features indicate a winning team.

In [ ]:
# REMOVING IRRELEVANT COLUMNS
df_model = df_wc.select_dtypes(exclude=['object'])
df_model = df_model.drop(columns=["Value", "Wage", "Nation_KitNumber", "ID", "Club_KitNumber"])

# FILLNA ON TARGET COLUMNS
df_model["Lifetime_WC_Matches"] = df_model["Lifetime_WC_Matches"].fillna(0)
df_model["Lifetime_WC_Matches"] = df_model["Lifetime_WC_Matches"].astype("int64")

df_model["Lifetime_WC_Matches_Bin"] = df_model["Lifetime_WC_Matches_Bin"].fillna(0)
df_model["Lifetime_WC_Matches_Bin"] = df_model["Lifetime_WC_Matches_Bin"].astype("int64")

df_model["Lifetime_WC_Matches_Bin4"] = df_model["Lifetime_WC_Matches_Bin4"].fillna(0)
df_model["Lifetime_WC_Matches_Bin4"] = df_model["Lifetime_WC_Matches_Bin4"].astype("int64")

df_model["Lifetime_WC_Matches_Bin5"] = df_model["Lifetime_WC_Matches_Bin5"].fillna(0)
df_model["Lifetime_WC_Matches_Bin5"] = df_model["Lifetime_WC_Matches_Bin5"].astype("int64")

df_model["Lifetime_WC_Matches_Bin10"] = df_model["Lifetime_WC_Matches_Bin10"].fillna(0)
df_model["Lifetime_WC_Matches_Bin10"] = df_model["Lifetime_WC_Matches_Bin10"].astype("int64")

# df_model.describe()

In [ ]:
# DROPPING ROWS FOR PLAYERS WHO HAVE NEVER APPEARED IN THE WORLD CUP
df_model = df_model.loc[df_model["Lifetime_WC_Matches_Bin"]==1]
# df_model.describe()

In [ ]:
# CORRELATION OF FEATURES FOR 2021 PLAYERS WHO HAVE APPEARED IN AT LEAST ONE WORLD CUP MATCH
corrMatrix = df_model.corr()
sn.heatmap(corrMatrix, annot = True)
# plt.figure(figsize = (25,25))
plt.show()

In [ ]:
lifetime_appearances = df_model["Lifetime_WC_Matches"]

# INVESTIGATION Y
lifetime_appearances.value_counts()
# lifetime_appearances.mean()
# lifetime_appearances.median()
# lifetime_appearances.mode()

# Player Features Modeling

In [ ]:
# # DEFINE X AND Y FOR THE MODEL
# # CHALLENGE IS COMPENSATING FOR THE UNBALANCED DATASET. 

# # ------- OPTION A ------- returns test score of 22.41% (MULTICLASS)
# # WHERE THE TARGET IS THE NUMBER OF LIFETIME WORLD CUP MATCHES PER 2021 PLAYER
# model_option_name = "Multiclass - All Lifetime WC Appearances"
# X = df_model.drop(["Lifetime_WC_Matches", "Lifetime_WC_Matches_Bin", 
#                    "Lifetime_WC_Matches_Bin4", "Lifetime_WC_Matches_Bin10", "Lifetime_WC_Matches_Bin5"], axis=1)
# y = df_model["Lifetime_WC_Matches"]

# ------- OPTION B ------- returns test score of 62.64%
# WHERE THE TARGET IS IF THE PLAYER HAS HAD MORE THAN 3 WC APPERANCES (295 OF 694, 42.5%)
model_option_name = "If players had 4 or more Lifetime WC Apperances"
X = df_model.drop(["Lifetime_WC_Matches", "Lifetime_WC_Matches_Bin", 
                   "Lifetime_WC_Matches_Bin4", "Lifetime_WC_Matches_Bin10", "Lifetime_WC_Matches_Bin5"], axis=1)
y = df_model["Lifetime_WC_Matches_Bin4"]

# # ------- OPTION B.2 ------- returns test score of 70.1%
# # WHERE THE TARGET IS IF THE PLAYER HAS HAD MORE THAN 4 WC APPERANCES (208 OF 694, 30.0%)
# model_option_name = "If players had 5 or more Lifetime WC Apperances"
# X = df_model.drop(["Lifetime_WC_Matches", "Lifetime_WC_Matches_Bin", 
#                    "Lifetime_WC_Matches_Bin4", "Lifetime_WC_Matches_Bin10", "Lifetime_WC_Matches_Bin5"], axis=1)
# y = df_model["Lifetime_WC_Matches_Bin5"]

# # ------- OPTION C ------- returns test score of 85.63%
# # WHERE THE TARGET IS IF THE PLAYER HAS HAD AT LEAST 9 WC APPERANCES (67 OF 694, 9.7%)
# model_option_name = "If players had 9 or more Lifetime WC Apperances"
# X = df_model.drop(["Lifetime_WC_Matches", "Lifetime_WC_Matches_Bin", 
#                    "Lifetime_WC_Matches_Bin4", "Lifetime_WC_Matches_Bin10", "Lifetime_WC_Matches_Bin5"], axis=1)
# y = df_model["Lifetime_WC_Matches_Bin10"]

# count
y.value_counts()

In [ ]:
# SPLIT AND TRAIN MODEL
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Target used: {model_option_name}")
print("-"*50)
print(f"Maximum X_train_scaled value: {X_train_scaled.max().round(4)}")
print(f"Minimum X_train_scaled value: {X_train_scaled.min().round(4)}")
print("-"*50)
print(f"Maximum X_test_scaled value: {X_test_scaled.max().round(4)}")
print(f"Minimum X_test_scaled value: {X_test_scaled.min().round(4)}")


In [ ]:
# RANDOM FOREST CLASSIFIER MODEL
clf = RandomForestClassifier(random_state=37, n_estimators=1000).fit(X_train_scaled, y_train)

print(f"Target used: {model_option_name}")
print("-"*50)
print(f'Training Score: {clf.score(X_train_scaled, y_train).round(4)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test).round(4)}')


In [ ]:
# IDENTIFYING FEATURE IMPORTANCE
features = clf.feature_importances_.round(4)
feature_names = list(X_train.columns.values)

In [ ]:
# PLOTTING THE FEATURE IMPORTANCE ANALYSIS

# MAKING A DATAFRAME OF THE FEATURES ANALYSIS OUTPUT
features_list = pd.Series(features)
featuresnames_list = pd.Series(feature_names)
features_df = pd.concat([featuresnames_list, features_list], axis = 1)
features_df = features_df.sort_values(by=1, ascending = True)
# features_df.rename(columns = {"0: Feature Name", "1: Importance Value"})

plt.figure(figsize = (25,25))
plt.title(f"Feature Importance: {model_option_name}", fontsize=25)
plt.xlabel("Importance", fontsize=22)
plt.xticks(fontsize=25)

plt.barh(range(len(features_df[1])), features_df[1], color="maroon", edgecolor="darkred")
plt.yticks(range(len(features_df[1])), features_df[0], fontsize = 20)
plt.show()

In [ ]:

# NUMBER OF TOP FEATURES TO INCLUDE
n = 5

# DATAFRAME FOR TOP FEATURES
top_features_df = features_df.sort_values(by=1, ascending = False).head(n)
# top_features_df

# CREATE LIST OF TOP N MOST IMPORTANT FEATURES
top_features_list = top_features_df[0].to_list()


In [ ]:
print(f"Target used: {model_option_name}")
print("-"*50)
print(f"Average Importance of All Features: {features.mean().round(4)}")
print(f"Maximum Importance of All Features: {features.max().round(4)}")
print(f"Minimum Importance of All Features: {features.min().round(4)}")
print("-"*50)

print(f"The Top {len(features_df.sort_values(by=1, ascending = False).head(n))} \
Most Important Features Impacting a Player's World Cup Apperance:")
print(f"{top_features_list}")

In [ ]:
# y_train.value_counts()
# y_test.value_counts()

# othe roptions: smote - synthetic data to up/down sample; precision recall metrics instead of test/train score

In [ ]:
# # SHAP PLOT

# # X, y = shap.datasets.adult()
# model = xgboost.XGBClassifier().fit(X, y)
# explainer = shap.TreeExplainer(model, X)
# shap_values = explainer(X)
# feature_names = [
#     a + ": " + str(b) for a,b in zip(X.columns, np.abs(shap_values.values).mean(0).round(2))
# ]

# shap.summary_plot(shap_values, X, max_display=X.shape[1], 
#                   feature_names=feature_names)

# # expecting that y is label encoded
# # change y to 1 or 0 , that the player appeared in the WC; if y = 0 then 0 else y = 1
# # alternative: multiclass classification; 
# # or could do regression (lin or ranfor) to predict continuous number

In [ ]:
# clf = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_train_scaled, y_train)
# print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
# print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')

In [ ]:
# feature_importances = clf.feature_importances_
# features = sorted(zip(X.columns, clf.feature_importances_), key = lambda x: x[1])
# cols = [f[0] for f in features]
# width = [f[1] for f in features]

# fig, ax = plt.subplots()

# fig.set_size_inches(10)
# plt.margins(y=0.001)

# ax.barh(y=cols, width=width)

# plt.show()

# create many features based on statistics per each country

In [ ]:
#only keep countries that are in the 2022 World Cup
data_df = data_df[(data_df['Country'].isin(teams_2022))]
data_df

In [ ]:
data_df = data_df.drop(columns = 'ID')

In [ ]:
data_df.columns
#which attribute features do we want out of the following column names?

In [ ]:
data_df['Nation_Position'].unique()

In [ ]:
# averaging stats of each player to get an overall 'score' for each team
average_for_teams = data_df.groupby(["Country"]).mean()
average_for_teams = average_for_teams.reset_index()
average_for_teams

In [ ]:
# median stats of each player to get an overall 'score' for each team
median_for_teams = data_df.groupby(["Country"]).median()
median_for_teams = median_for_teams.reset_index()
len(median_for_teams)

In [ ]:
# median stats of each player to get an overall 'score' for each team
max_for_teams = data_df.groupby(["Country"]).max()
max_for_teams = max_for_teams.reset_index()
len(max_for_teams)

In [ ]:
# min stats of each player to get an overall 'score' for each team
min_for_teams = data_df.groupby(["Country"]).min()
min_for_teams = min_for_teams.reset_index()
len(min_for_teams)

In [ ]:
std_for_teams = data_df.groupby(["Country"]).std()
std_for_teams = std_for_teams.reset_index()
len(std_for_teams)

#Features showing only certain positions: Goalie, for example

In [ ]:
#only goalies
goalies = data_df.loc[data_df['Nation_Position'] == 'GK']
len(goalies)

In [ ]:
#we only have 20 known goalies available
max_for_goalies = goalies.groupby("Country").max()
max_for_goalies = max_for_goalies.reset_index()
max_for_goalies

In [ ]:
data_df['PreferredPositions'].unique()

In [ ]:
#only goalies
goalies2 = data_df.loc[data_df['PreferredPositions'] == 'GK']
len(goalies2)

In [ ]:
max_for_goalies2 = goalies2.groupby("Country").max()
max_for_goalies2 = max_for_goalies2.reset_index()
max_for_goalies2

# Dataset does not identify the position each player will play in the world cup yet. However, we can look at preferred position to get a clue.

In [ ]:
data_df['Nation_Position'].value_counts()
# data_df['Nation_Position'].count()

# SQL Database Connection

In [ ]:
# Connect to local database

# protocol = 'postgresql'
# username = 'postgres'
# password = 'admin'
# host = 'localhost'
# port = 5432
# database_name = 'players_db'
# rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
# engine = create_engine(rds_connection_string)
# rds_connection_string

In [ ]:
# Checking Tables Names in PgAdmin

# engine.table_names()

In [ ]:
# player_info_df.to_sql(name='player_info', con=engine, if_exists='replace', index=False)

In [ ]:
# matches_df.to_sql(name='matches', con=engine, if_exists='replace', index=False)

In [ ]:
# averages_for_teams.to_sql(name='team_averages', con=engine, if_exists='replace', index=False)